In [17]:
import os, json, pandas as pd, requests
from datetime import date, datetime, timezone, timedelta
from urllib.parse import urlencode
from dotenv import load_dotenv

# folders
RAW_DIR = "../data/raw"
os.makedirs(RAW_DIR, exist_ok=True)

# env (for GIE)
load_dotenv()
GIE_API_KEY = os.getenv("GIE_API_KEY", "").strip()

def ts():
    return datetime.now(tz=timezone.utc).strftime("%Y%m%dT%H%M%SZ")

# GIE: AGSI (gas storage) raw dump

In [18]:
AGSI_BASE = "https://agsi.gie.eu/api"

def agsi_dump_raw(params: dict, out_name: str):
    """Dump AGSI endpoint with simple paging to Parquet."""
    assert GIE_API_KEY, "Set GIE_API_KEY in your .env"
    headers = {"x-key": GIE_API_KEY}
    frames, page = [], 1
    while True:
        q = dict(params); q.update({"page": page, "size": 1000})
        r = requests.get(AGSI_BASE, headers=headers, params=q, timeout=60)
        if r.status_code == 429:
            # gentle retry
            import time; time.sleep(65); r = requests.get(AGSI_BASE, headers=headers, params=q, timeout=60)
        r.raise_for_status()
        payload = r.json()
        data = payload.get("data", [])
        if not data: break
        frames.append(pd.DataFrame.from_records(data))
        if page >= int(payload.get("last_page", page)): break
        page += 1
    df = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
    path = os.path.join(RAW_DIR, f"{out_name}_{ts()}.parquet")
    df.to_parquet(path, index=False)
    print(f"AGSI saved → {path} | rows={len(df)}")
    return path, df

# Examples (pick one or both)
# EU aggregate
agsi_eu_path, agsi_eu_df = agsi_dump_raw({"type": "eu"}, out_name="agsi_eu")

AGSI saved → ../data/raw/agsi_eu_20250820T092136Z.parquet | rows=5344


In [19]:
ALSI_BASE = "https://alsi.gie.eu/api"

def alsi_dump_raw(params: dict, out_name: str):
    assert GIE_API_KEY, "Set GIE_API_KEY in your .env"
    headers = {"x-key": GIE_API_KEY}
    frames, page = [], 1
    while True:
        q = dict(params); q.update({"page": page, "size": 1000})
        r = requests.get(ALSI_BASE, headers=headers, params=q, timeout=60)
        if r.status_code == 429:
            import time; time.sleep(65); r = requests.get(ALSI_BASE, headers=headers, params=q, timeout=60)
        r.raise_for_status()
        payload = r.json()
        data = payload.get("data", [])
        if not data: break
        frames.append(pd.DataFrame.from_records(data))
        if page >= int(payload.get("last_page", page)): break
        page += 1
    df = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
    path = os.path.join(RAW_DIR, f"{out_name}_{ts()}.parquet")
    df.to_parquet(path, index=False)
    print(f"ALSI saved → {path} | rows={len(df)}")
    return path, df

# EU aggregate
alsi_eu_path, alsi_eu_df = alsi_dump_raw({"type": "eu"}, out_name="alsi_eu")


ALSI saved → ../data/raw/alsi_eu_20250820T092205Z.parquet | rows=4979


In [20]:
import os, time, pandas as pd
from urllib.parse import urlencode
from datetime import datetime, date, timedelta

ENTSOG_BASE = "https://transparency.entsog.eu/api/v1"

def _daterange_chunks(start_date: date, end_date: date, chunk_days: int):
    cur = start_date
    while cur <= end_date:
        nxt = min(cur + timedelta(days=chunk_days-1), end_date)
        yield cur, nxt
        cur = nxt + timedelta(days=1)

def _read_csv_retry(url, tries=3, sleep=1.5):
    for k in range(tries):
        try:
            return pd.read_csv(url)
        except Exception as e:
            if k == tries - 1:
                raise
            time.sleep(sleep * (2**k))

def entsog_store_aggregated_physical_flow_csv_chunked(start: str, end: str,
                                                      chunk_days: int = 14) -> str:
    """Robustly pull aggregated country-level Physical Flow with chunking (avoids 502)."""
    start_d = datetime.fromisoformat(start).date()
    end_d   = datetime.fromisoformat(end).date()

    frames = []
    for a, b in _daterange_chunks(start_d, end_d, chunk_days):
        params = {"indicator": "Physical Flow",
                  "from": a.isoformat(),
                  "to": b.isoformat(),
                  "periodType": "day",
                  "limit": 20000}
        url = f"{ENTSOG_BASE}/aggregatedData.csv?{urlencode(params)}"
        df = _read_csv_retry(url, tries=4, sleep=2.0)
        if not df.empty:
            frames.append(df)

    if not frames:
        raise RuntimeError("No aggregatedData rows returned across all chunks.")

    # Safe concat + store raw
    all_cols = list({c for f in frames for c in f.columns})
    frames = [f.reindex(columns=all_cols, copy=False) for f in frames]
    df_all = pd.concat(frames, ignore_index=True, copy=False)

    out = os.path.join("data_raw", f"entsog_aggregated_physical_flow_{start}_{end}.parquet")
    df_all.to_parquet(out, index=False)
    print(f"ENTSOG aggregated (country) saved → {out} | rows={len(df_all)}")
    return out
END = date.today().isoformat()
START = (date.today() - timedelta(days=180)).isoformat()
path_country = entsog_store_aggregated_physical_flow_csv_chunked(START, END, chunk_days=14)


ENTSOG aggregated (country) saved → data_raw/entsog_aggregated_physical_flow_2025-02-21_2025-08-20.parquet | rows=61655


In [21]:
from meteostat import Point, Daily

def meteostat_hdd_dump(cities, start_dt, end_dt, base_c=18.0, out_name="meteostat_hdd"):
    frames = []
    for (name, lat, lon, alt) in cities:
        p = Point(lat, lon, alt)
        df = Daily(p, start_dt, end_dt).fetch()
        if df.empty: 
            continue
        if "tavg" not in df.columns:
            df["tavg"] = (df.get("tmin") + df.get("tmax")) / 2.0
        out = df[["tavg"]].copy()
        out["city"] = name
        out = out.reset_index().rename(columns={"time": "date"})
        frames.append(out)
    panel = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
    path = os.path.join(RAW_DIR, f"{out_name}_{start_dt.date()}_{end_dt.date()}_{ts()}.parquet")
    panel.to_parquet(path, index=False)
    print(f"Meteostat HDD (raw temps) saved → {path} | rows={len(panel)}")
    return path, panel

# Your base cities
cities = [
    ("Amsterdam", 52.37, 4.90, 13),
    ("Berlin",    52.52, 13.40, 34),
    ("Paris",     48.86, 2.35, 35),
    ("Milan",     45.46, 9.19, 120),
    ("Madrid",    40.42, -3.70, 667),
    # add more later if you want
]

meteostat_path, meteostat_df = meteostat_hdd_dump(
    cities,
    start_dt = datetime.now() - timedelta(days=180),
    end_dt   = datetime.now()
)


Meteostat HDD (raw temps) saved → ../data/raw/meteostat_hdd_2025-02-21_2025-08-20_20250820T092255Z.parquet | rows=900
